<a href="https://colab.research.google.com/github/syafie-nzm/spotify-weekly-chart/blob/main/scrap_streams_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install requests beautifulsoup4

In [ ]:
!pip install lxml


In [ ]:
import requests
#Spotify Weekly Chart - Malaysia - 2023/10/12
url = 'https://kworb.net/spotify/country/my_weekly.html'
raw_data = requests.get(url)

#untuk inspect element dalam html tu dulu before kita extract information
print(raw_data.text)

In [ ]:
from bs4 import BeautifulSoup
#store kat list ni
data = []

soup = BeautifulSoup(raw_data.text, 'html.parser')

#search all row (element tr) yang ada info kita nak, based on inspection kita
#tadi setiap entry dalam chart tu akan start dengan tr
entries = soup.find_all('tr')

for entry in entries:

  #maklumat yang kita nak extract in this each entry (element td)
  columns = entry.find_all('td')

  #special case, first entry tu baru header dia, takmau
  if len(columns) == 0:
    continue

  position = columns[0].text.strip()
  change = columns[1].text.strip()

  #artist n title dalam satu columns, boleh splitkan
  artist_title_element = columns[2].text.strip()
  artist, title = artist_title_element.split('-', 1)

  weeks = columns[3].text.strip()
  peak = columns[4].text.strip()
  streams = columns[6].text.strip()
  streams_plus = columns[7].text.strip()
  total = columns[8].text.strip()

  data.append({'Position': position,
              'Change': change,
              'Artist': artist,
              'Title': title,
              'Weeks': weeks,
              'Peak': peak,
              'Streams': streams,
              'Streams+': streams_plus,
              'Total': total})


In [ ]:
for entry in data:
  print(entry)

{'Position': '1', 'Change': '=', 'Artist': 'Jung Kook ', 'Title': ' Seven (w/ Latto)', 'Weeks': '13', 'Peak': '1', 'Streams': '668,031', 'Streams+': '-28,878', 'Total': '11,408,698'}
{'Position': '2', 'Change': '=', 'Artist': 'Ghea Indrawari ', 'Title': ' Jiwa Yang Bersedih', 'Weeks': '12', 'Peak': '2', 'Streams': '558,172', 'Streams+': '-53,385', 'Total': '6,204,670'}
{'Position': '3', 'Change': '+2', 'Artist': 'Tate McRae ', 'Title': ' greedy', 'Weeks': '4', 'Peak': '3', 'Streams': '552,650', 'Streams+': '+90,568', 'Total': '1,614,641'}
{'Position': '4', 'Change': '-1', 'Artist': 'Jung Kook ', 'Title': ' 3D (w/ Jack Harlow)', 'Weeks': '2', 'Peak': '3', 'Streams': '515,488', 'Streams+': '+18,409', 'Total': '1,012,567'}
{'Position': '5', 'Change': '-1', 'Artist': 'Anggi Marito ', 'Title': ' Tak Segampang Itu', 'Weeks': '30', 'Peak': '1', 'Streams': '462,505', 'Streams+': '-25,774', 'Total': '16,036,220'}
{'Position': '6', 'Change': '+13', 'Artist': 'Mitski ', 'Title': ' My Love Mine Al

In [ ]:
import csv

csv_path = '/content/drive/MyDrive/raw_spotify.csv'

with open(csv_path, 'w', newline='') as csvfile:
    fieldnames = data[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in data:
        writer.writerow(row)

print(f'Data written to {csv_path}')

Data written to /content/drive/MyDrive/raw_spotify.csv


In [45]:
import pandas as pd
csv_path = '/content/drive/MyDrive/raw_spotify.csv'
df = pd.read_csv(csv_path)


In [46]:
df

,Position,Change,Artist,Title,Weeks,Peak,Streams,Streams+,Total
0,1,=,Jung Kook,Seven (w/ Latto),13,1,"668,031","-28,878","11,408,698"
1,2,=,Ghea Indrawari,Jiwa Yang Bersedih,12,2,"558,172","-53,385","6,204,670"
2,3,+2,Tate McRae,greedy,4,3,"552,650","+90,568","1,614,641"
3,4,-1,Jung Kook,3D (w/ Jack Harlow),2,3,"515,488","+18,409","1,012,567"
4,5,-1,Anggi Marito,Tak Segampang Itu,30,1,"462,505","-25,774","16,036,220"
...,...,...,...,...,...,...,...,...,...
195,196,RE,Projector Band,Sudah Ku Tahu,67,7,"91,798",NaN,"3,299,807"
196,197,-24,Lokman Naufal,Haruman Terindah,5,157,"91,723","-2,739","478,821"
197,198,NEW,NCT 127,Fact Check,1,198,"91,710",NaN,"91,710"
198,199,-24,H.E.R.,Best Part (w/ Daniel Caesar),195,67,"91,697","-2,392","10,162,247"


In [47]:
print(df.dtypes)

Position     int64
Change      object
Artist      object
Title       object
Weeks        int64
Peak         int64
Streams     object
Streams+    object
Total       object
dtype: object


In [48]:
columns_to_remove = ['Streams+', 'Change']
df = df.drop(columns=columns_to_remove)
print(df.dtypes)

Position     int64
Artist      object
Title       object
Weeks        int64
Peak         int64
Streams     object
Total       object
dtype: object


In [49]:
df['Streams'] = df['Streams'].str.replace(',', '').astype(int)
df['Total'] = df['Total'].str.replace(',', '').astype(int)
print(df.dtypes)

Position     int64
Artist      object
Title       object
Weeks        int64
Peak         int64
Streams      int64
Total        int64
dtype: object


In [50]:
#make it top 50 only
df = df.drop(index=range(50,200))
df

,Position,Artist,Title,Weeks,Peak,Streams,Total
0,1,Jung Kook,Seven (w/ Latto),13,1,668031,11408698
1,2,Ghea Indrawari,Jiwa Yang Bersedih,12,2,558172,6204670
2,3,Tate McRae,greedy,4,3,552650,1614641
3,4,Jung Kook,3D (w/ Jack Harlow),2,3,515488,1012567
4,5,Anggi Marito,Tak Segampang Itu,30,1,462505,16036220
5,6,Mitski,My Love Mine All Mine,3,6,403921,861068
6,7,JENNIE,You & Me,1,7,370517,370517
7,8,Taylor Swift,Cruel Summer,46,6,360966,8748330
8,9,SZA,Kill Bill,44,1,359867,25301351
9,10,NewJeans,Super Shy,14,2,354914,7566453


In [51]:
save_path = '/content/drive/MyDrive/cleaned_spotify.csv'
df.to_csv(save_path, index=False)

In [1]:
!git config --global user.email “syafie.nzm@gmail.com”
!git config --global user.name “syafie-nzm”

In [2]:
!git clone https://github.com/syafie-nzm/spotify-weekly-chart.git

Cloning into 'spotify-weekly-chart'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 4 (delta 1), pack-reused 0
Receiving objects: 100% (18/18), 63.08 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
cd /content/spotify-weekly-chart

/content/spotify-weekly-chart


In [4]:
!cp /content/drive/MyDrive/cleaned_spotify.csv .

In [5]:
!git add .
!git commit -m 'Added my file'
!git push https://<secrettoken>@github.com/syafie-nzm/spotify-weekly-chart.git

[main f6547f8] Added my file
 1 file changed, 51 insertions(+)
 create mode 100644 cleaned_spotify.csv
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 1.89 KiB | 1.89 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: Bypassed rule violations for refs/heads/main:
remote: 
remote: - Changes must be made through a pull request.
remote: 
To https://github.com/syafie-nzm/spotify-weekly-chart.git
   cff11be..f6547f8  main -> main
